In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
%pip install spacy
!python -m spacy download en_core_web_sm


Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.4 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

# Function to transform gendered pronouns
def transform_pronouns(sentence, target_gender):
    male_to_female = {
        "he": "she",
        "him": "her",
        "his": "her",
        "himself": "herself"
    }
    female_to_male = {
        "she": "he",
        "her": "his", # Changed from 'him' to 'his' for possessive cases
        "hers": "his",
        "herself": "himself"
    }

    # Special case dictionary for handling context-dependent 'her'
    # Based on dependency labels in the parse tree
    special_cases = {
        "female_to_male": {
            "poss": {"her": "his"}, # Possessive 'her' -> 'his'
            "dobj": {"her": "him"}, # Direct object 'her' -> 'him'
            "dative": {"her": "him"}, # Indirect object 'her' -> 'him'
            "pobj": {"her": "him"} # Object of preposition 'her' -> 'him'
        }
    }

    mapping = male_to_female if target_gender == "female" else female_to_male
    spec_cases = special_cases.get("female_to_male", {}) if target_gender == "male" else {}

    doc = nlp(sentence)
    transformed_tokens = []

    for token in doc:
        lower_token = token.text.lower()
        # Special handling for 'her' when target is male, based on dependency
        if target_gender == "male" and lower_token == "her" and token.dep_ in spec_cases:
            transformed_token = spec_cases[token.dep_].get(lower_token, mapping.get(lower_token, token.text))
        elif lower_token in mapping:
            transformed_token = mapping[lower_token]
        else:
            transformed_token = token.text

        # Preserve capitalization
        if token.text[0].isupper() and transformed_token[0].islower():
            transformed_token = transformed_token.capitalize()
            
        transformed_tokens.append(transformed_token)

    # Join tokens without adding extra spaces
    result = ""
    for i, token in enumerate(transformed_tokens):
        if i > 0:
            # Add space before token if it's not punctuation
            if not (token in ".,;!?:'\")" or transformed_tokens[i-1].endswith("'")):
                result += " "
        result += token
        
    return result

# Example usage
examples = [
    "He gave her his book.",
    "She gave him her book.",
    "This is her book.",
    "Give the book back to her.",
    "He is proud of his achievements.",
    "She is proud of her achievements.",
    "He will do it himself.",
    "She will do it herself.",
    "He is a good friend.",
    "She is a good friend.",
    "He is a doctor.",
    "She is a doctor.",
    "He is a teacher.",
    "She is a teacher.",
    "He is a student.",
    "She is a student.",
    "He is a parent.",
    "She is a parent.",
    "He is a leader."
]

print("Example Transformations:")
target_gender = "female"
for t in examples:
    transformed_sentence = transform_pronouns(t, target_gender)
    print(f"Original: {t}")
    print(f"Transformed: {transformed_sentence}")
    print()

# Function to calculate accuracy based on matching transformed output to expected output
def calculate_accuracy(test_cases):
    correct = 0
    total = len(test_cases)
    
    for original, gender, expected in test_cases:
        transformed = transform_pronouns(original, gender)
        if transformed == expected:
            correct += 1
            result = "Correct"
        else:
            result = "Incorrect"
        
        print(f"Original: {original}")
        print(f"Target Gender: {gender}")
        print(f"Predicted: {transformed}")
        print(f"Expected: {expected}")
        print(f"Result: {result}")
        print()
    
    accuracy = (correct / total) * 100
    return accuracy

# Test cases
test_cases = [
    ("He is going to the market.", "female", "She is going to the market."),
    ("His book is on the table.", "female", "Her book is on the table."),
    ("I saw him yesterday.", "female", "I saw her yesterday."),
    ("He hurt himself.", "female", "She hurt herself."),
    ("I called him last night.", "female", "I called her last night."),
    ("That is his car.", "female", "That is her car."),
    ("He told me about his trip.", "female", "She told me about her trip."),
    ("The teacher gave him a warning.", "female", "The teacher gave her a warning."),
    ("He blames himself for the mistake.", "female", "She blames herself for the mistake."),
    ("He brought his laptop.", "female", "She brought her laptop."),
    ("He made it himself.", "female", "She made it herself."),
    ("I don't like his attitude.", "female", "I don't like her attitude."),
    ("Tell him to come here.", "female", "Tell her to come here."),
    ("She is going to the market.", "male", "He is going to the market."),
    ("Her book is on the table.", "male", "His book is on the table."),
    ("I saw her yesterday.", "male", "I saw him yesterday."),
    ("She hurt herself.", "male", "He hurt himself."),
    ("I called her last night.", "male", "I called him last night."),
    ("That is her car.", "male", "That is his car."),
    ("She told me about her trip.", "male", "He told me about his trip."),
    ("The teacher gave her a warning.", "male", "The teacher gave him a warning."),
    ("She blames herself for the mistake.", "male", "He blames himself for the mistake."),
    ("She brought her laptop.", "male", "He brought his laptop."),
    ("She made it herself.", "male", "He made it himself."),
    ("I don't like her attitude.", "male", "I don't like his attitude."),
    ("Tell her to come here.", "male", "Tell him to come here.")
]

print("\n\nEvaluating Test Cases:")
accuracy = calculate_accuracy(test_cases)
print(f"\nAccuracy on test cases: {accuracy:.2f}%")

Example Transformations:
Original: He gave her his book.
Transformed: She gave her her book.

Original: She gave him her book.
Transformed: She gave her her book.

Original: This is her book.
Transformed: This is her book.

Original: Give the book back to her.
Transformed: Give the book back to her.

Original: He is proud of his achievements.
Transformed: She is proud of her achievements.

Original: She is proud of her achievements.
Transformed: She is proud of her achievements.

Original: He will do it himself.
Transformed: She will do it herself.

Original: She will do it herself.
Transformed: She will do it herself.

Original: He is a good friend.
Transformed: She is a good friend.

Original: She is a good friend.
Transformed: She is a good friend.

Original: He is a doctor.
Transformed: She is a doctor.

Original: She is a doctor.
Transformed: She is a doctor.

Original: He is a teacher.
Transformed: She is a teacher.

Original: She is a teacher.
Transformed: She is a teacher.

Ori